<a href="https://colab.research.google.com/github/zynkchen/Animagine-XL-Lightning-Colab/blob/main/cartoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

环境初始化，磁盘准备...

In [ ]:
# 1. 挂载 Google Drive
from google.colab import drive
import os

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. 定义路径
# 本地临时缓存路径（Hugging Face 默认位置）
local_cache_path = "/root/.cache/huggingface/hub"
# 云盘持久化大文件路径
drive_model_path = "/content/drive/MyDrive/AI_Models_Cache/hub"

os.makedirs("/content/drive/MyDrive/AI_Models_Cache", exist_ok=True)
# 定义图片保存目录 output_dir
output_dir = "/content/drive/MyDrive/AI_Models_Cache/AI_Pony_Output"
os.makedirs(output_dir, exist_ok=True)

# 3. 执行“嫁接”逻辑
if not os.path.exists(local_cache_path):
    # 先创建父级目录
    os.makedirs(os.path.dirname(local_cache_path), exist_ok=True)
    # 如果云盘已有备份，则创建软链接映射
    if os.path.exists(drive_model_path):
        print("🔗 发现云盘已有模型缓存，正在建立连接...")
        os.symlink(drive_model_path, local_cache_path)
    else:
        # 如果是第一次运行，创建云盘目录并链接
        print("📁 第一次运行，正在初始化云盘存储空间...")
        os.makedirs(drive_model_path, exist_ok=True)
        os.symlink(drive_model_path, local_cache_path)

# 4. 安装环境（依然安装在 Colab 临时空间，保证运行速度）
print("⚡ 正在安装核心库至临时空间...")
# 强制同步所有核心库版本，消除冲突提示
!pip install -q -U torch torchvision torchaudio diffusers transformers accelerate peft safetensors xformers

print(f"✅ 配置完成！\n🚀 大文件将自动同步至: {drive_model_path}\n🚀 临时库和元数据保留在 Colab 内存加速。")
#@title Keep Alive for Mobile Users
from IPython.display import Audio,display
import numpy as np
# 这是一个防止 Colab 断开连接的音频播放器
display(Audio(np.array([0] * 2 * 3600 * 3000, dtype=np.int8), normalize=False, rate=3000, autoplay=True))

运行环境准备...

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, EulerDiscreteScheduler

base_model = "cagliostrolab/animagine-xl-3.1"
lora_model = "ByteDance/SDXL-Lightning"

print("🚀 重新尝试加载引擎（采用更稳健的加载策略）...")

try:
    # 1. 加载主模型
    # 删除了 device_map="auto"，改为显式加载后 .to("cuda")
    pipe = StableDiffusionXLPipeline.from_pretrained(
        base_model,
        torch_dtype=torch.float16,
        use_safetensors=True,
        low_cpu_mem_usage=True
    )

    # 2. 将模型移至 GPU
    pipe.to("cuda")

    print("📥 加载加速插件 (Lightning LoRA)...")
    # 加载 LoRA
    pipe.load_lora_weights(lora_model, weight_name="sdxl_lightning_4step_lora.safetensors")
    # 融合 LoRA 权重以提高推理速度
    pipe.fuse_lora()

    # 3. 调度器配置
    pipe.scheduler = EulerDiscreteScheduler.from_config(
        pipe.scheduler.config,
        timestep_spacing="trailing"
    )

    # 4. 开启极限显存优化
    pipe.enable_xformers_memory_efficient_attention()
    pipe.enable_vae_tiling()

    print("✅ 引擎就绪！模型已成功部署在 GPU。")

except Exception as e:
    print(f"❌ 再次失败，错误详情: {e}")
    print("\n💡 最终排查方案：如果依然报错，请在单元 1 中取消 Google Drive 软链接，直接下载到 Colab 本地空间测试。")

一次生成4张图

核心状态与姿势（Core Tags）

    基础触发：nsfw, erotic, suggestive, naked, nude

    身体细节：detailed anatomy, realistic skin, skin texture, soft lighting, nipples, pussy, cleavage

    动作/状态：spreading, legs apart, from behind, bent over, looking at viewer, blush, sweating


In [ ]:
import uuid
import random
from IPython.display import display, HTML, update_display
from PIL import Image
import io
import base64

# ================= 创作控制面板 =================
# @markdown ### 🟢 提示词设置 (Prompt)
prompt = "score_9, score_8_up, score_7_up, source_anime, nsfw, 1girl,  masterpiece, highres, (realistic skin:1.2), (detailed anatomy:1.1), cinemetic lighting, depth of field, erotic, suggestive, naked" # @param {type:"string"}
negative_prompt = "lowres,  deformed, disfigured,lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry, artist name" # @param {type:"string"}

# @markdown ### 🟡 画面参数 (Parameters)
num_images = 4 # @param [1, 2, 3, 4] {type:"raw"}
width = 832 # @param [512, 768, 832, 1024, 1216] {allow-input: true}
height = 1216 # @param [512, 768, 832, 1024, 1216] {allow-input: true}
steps = 13 # @param {type:"slider", min:1, max:20, step:1}
cfg_scale = 1.7 # @param {type:"slider", min:0.0, max:4.0, step:0.1}

# @markdown ### 🔵 随机性控制 (Seed)
seed = -1 # @param {type:"integer"}

# ================= 实时布局逻辑 =================

# 1. 初始化 HTML 网格容器
grid_id = str(uuid.uuid4())
grid_html = f"""
<div id="{grid_id}" style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px; width: 700px;">
    {'<div style="aspect-ratio: ' + str(width/height) + '; background: #333; display: flex; align-items: center; justify-content: center; color: #666; font-family: sans-serif;">Wait...</div>' * num_images}
</div>
"""
display_handle = display(HTML(grid_html), display_id=grid_id)

def img_to_base64(img):
    buffered = io.BytesIO()
    img.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode()

positive_prefix = "masterpiece, best quality, "
all_images_html = [""] * num_images

for i in range(num_images):
    current_seed = random.randint(0, 1000000) if seed == -1 else seed + i
    generator = torch.Generator("cuda").manual_seed(current_seed)

    # 执行推理
    with torch.inference_mode():
        image = pipe(
            prompt = positive_prefix + prompt,
            negative_prompt = negative_prompt,
            num_inference_steps = steps,
            guidance_scale = cfg_scale,
            width = width,
            height = height,
            generator = generator
        ).images[0]

    # 保存原图
    save_path = f"{output_dir}/anim_s{current_seed}.png"
    image.save(save_path)

    # 准备缩略图预览
    preview_size = (int(image.width / 3), int(image.height / 3))
    preview_img = image.resize(preview_size)
    b64_str = img_to_base64(preview_img)

    # 更新 HTML 内容
    all_images_html[i] = f'<div style="text-align:center;"><img src="data:image/png;base64,{b64_str}" style="width:100%; border-radius:4px;"><p style="color:#888; font-size:12px;">Seed: {current_seed}</p></div>'

    # 实时刷新显示
    current_grid = f"""
    <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px; width: 700px;">
        {''.join([img_html if img_html else '<div style="aspect-ratio: ' + str(width/height) + '; background: #333; display: flex; align-items: center; justify-content: center; color: #999;">Generating...</div>' for img_html in all_images_html])}
    </div>
    """
    update_display(HTML(current_grid), display_id=grid_id)

print(f"✅ 生成完毕！所有高清图已入库。")